In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models

import time
import os.path as osp
from PIL import Image
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline  

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [ ]:
class_data_path = '/data/skbharti/Fine-ProtoNet/data'
MODEL_PATH = 'models/proto_model.pt'
PROTOTYPES_PATH = 'models/prototypes.pt'


rt={'birds'    : osp.join(class_data_path,'class_data/CUB_200_2011'),
    'dogs'     : osp.join(class_data_path,'class_data/dogs'),
    'aircrafts': osp.join(class_data_path,'class_data/aircrafts'),
    'cars'     : osp.join(class_data_path,'class_data/cars')
   }
id_class_dict = {}
        
file_path1 = osp.join(rt['birds'],'classes.txt')
lines1 = [(line.strip()).split(" ",1) for line in open(file_path1, 'r').readlines()]
        
file_path2 = osp.join(rt['dogs'],'classes.txt')
lines2 = [(line.strip()).split(" ",1) for line in open(file_path2, 'r').readlines()]
        
file_path3 = osp.join(rt['aircrafts'],'classes.txt')
lines3 = [(line.strip()).split(" ",1) for line in open(file_path3, 'r').readlines()]
        
file_path4 = osp.join(rt['cars'],'classes.txt')
lines4 = [(line.strip()).split(" ",1) for line in open(file_path4, 'r').readlines()]
        
mat = lines1 + lines2 +lines3 + lines4
total_subclasses = len(mat)

new_mat = [] 

lines1 = None
del lines1
lines2 = None
del lines2
lines3 = None
del lines3
lines4 = None
del lines4

max1=0
max2=0
max3=0
max4=0

def coarseclass_name(x): #x is id>=1
    if (x<=max1):
        return "Birds"
    elif (x<=max2):
        return "Dogs"
    elif (x<=max3):
        return "Aircrafts"
    elif (x<=max4):
        return "Cars"
    
def subclass_name(x): # x is id >=1
    class_name = new_mat[x-1]
    return class_name


In [3]:
img_height, img_width, img_channels = 318, 318, 3

class Dataset2(Dataset):
    
    def __init__(self,data_type,classes):  
        
        
        global new_mat
        new_mat = []
        data_leng = []
        #all_data = [[ ]]
        for i in range(len(classes)):
            file_path = osp.join(rt[classes[i]],'img_label_train_test.txt')
            line = [(line.strip()).split() for line in open(file_path, 'r').readlines()]
            if(i!=0):
                all_data = all_data + line
            else:
                all_data = line
            data_leng.append(len(all_data))
    
        
        tmp_tdata =[]
        tmp_tlabel =[]
        tmp_tstdata =[]
        tmp_tstlabel =[]
        tmp_vdata =[]
        tmp_vlabel =[]
        
        j=0
        for i in range(len(all_data)):
            for k in range(len(data_leng)):
                if i< data_leng[k]:
                    root = rt[classes[k]]
                    break
                    
            if int(all_data[i][2])==1:
                
                path =  osp.join(root,'images',str(all_data[i][0]))
                j = j+1
                if(j%4!= 0):
                    tmp_tdata.append(path)
                    tmp_tlabel.append(int(all_data[i][1]))
                else:
                    tmp_vdata.append(path)
                    tmp_vlabel.append(int(all_data[i][1]))
                    
                    
            elif int(all_data[i][2])==0:
                path =  osp.join(root,'images',str(all_data[i][0]))
                tmp_tstdata.append(path)
                tmp_tstlabel.append(int(all_data[i][1]))
               
                
                
        k_min = 20       
        lb=1
        unq = np.unique(tmp_tlabel)
        for i in unq:  
            ind = np.argwhere(np.array(tmp_tlabel) == i).reshape(-1)   
            ind.sort()
            if (len(ind)<k_min):
                
                p=0
                for j in ind:
                    del tmp_tlabel[j-p]
                    del tmp_tdata[j-p]
                    p=p+1
                    
                tstind = np.argwhere(np.array(tmp_tstlabel) == i).reshape(-1) 
                tstind.sort()
                p=0
                for j in tstind:
                    del tmp_tstlabel[j-p]
                    del tmp_tstdata[j-p]
                    p=p+1
                    
                vind = np.argwhere(np.array(tmp_vlabel) == i).reshape(-1) 
                vind.sort()
                p=0
                for j in vind:
                    del tmp_vlabel[j-p]
                    del tmp_vdata[j-p]
                    p=p+1
                    
            else:
                global mat
                new_mat.append(mat[i-1][1])
                if(i!=lb):
                    
                    for k in ind:
                        tmp_tlabel[k]=lb
                    
                    tstind = np.argwhere(np.array(tmp_tstlabel) == i).reshape(-1) 
                    tstind.sort()
                    for k in tstind:
                        tmp_tstlabel[k]=lb
                
                    vind = np.argwhere(np.array(tmp_vlabel) == i).reshape(-1) 
                    vind.sort()
                    for k in vind:
                        tmp_vlabel[k] = lb
                
                if i<=200:  # make sure order birds, dogs, aircrafts, cars
                    global max1
                    max1= lb
                    
                elif i<=320:
                    global max2
                    max2= lb
                    
                elif i<=390:
                    global max3
                    max3= lb
                        
                elif i<=586:
                    global max4
                    max4= lb
                    
                lb = lb+1
                
                
        l1,l2,l3,l4=0,0,0,0 
        uq,count = np.unique(tmp_tlabel, return_counts = True)
        for i,u in enumerate(uq):
            if(u<=max1):
                l1= l1+ count[i]
            elif(u<=max2):
                l2= l2+ count[i]
            elif(u<=max3):
                l3= l3+ count[i]
            elif(u<=max4):
                l4= l4+ count[i]
                
                
        #print(max1,max2,max3,max4)   
        #print(len(np.unique(tmp_tlabel)))
        
        leng =[]     
        if(l1!=0):
            leng.append(l1)
        if(l2!=0):
            leng.append(l2)
        if(l3!=0):
            leng.append(l3)
        if(l4!=0):
            leng.append(l4)
        for i in range(len(leng)-1):
            leng[i+1]= leng[i+1] + leng[i]
            
        self.leng = leng
        
        if data_type == "train":
            self.data = tmp_tdata 
            self.label = tmp_tlabel        
            self.length = len(np.unique(tmp_tlabel))
            
        elif data_type == "val":
            self.data = tmp_vdata 
            self.label = tmp_vlabel
            self.length = len(np.unique(tmp_vlabel))
            
        if data_type == "test":
            self.data = tmp_tstdata 
            self.label = tmp_tstlabel
            self.length = len(np.unique(tmp_tstlabel))
        
        #random horizontal flip can be added
        self.transform = transforms.Compose([
            transforms.Resize((318,318)), 
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return self.length

    def __getitem__(self, i):
        path, label = self.data[i], self.label[i]
        image = self.transform(Image.open(path).convert('RGB'))  
        return image, label


In [4]:
import torch
import numpy as np


class Batch_Sampler():

    def __init__(self, label, leng, n_cls, n_per,n_batch):
        self.n_batch = n_batch
        self.n_cls = n_cls   
        self.n_per = n_per
        self.leng = leng

        label = np.array(label)
        self.m_ind = {}  
        length=[]
        length.append(0)
        for i in leng:
            length.append(i)
        #print("batch",length)
    
        labels = {}
        for i in range(len(leng)):
            labels[str(i+1)] = label[length[i]:length[i+1]]
        
        self.m_ind={}
        
        for j in range(len(leng)):
            indx=[]
            for i in np.unique(labels[str(j+1)]):  
                ind = np.argwhere(label == i).reshape(-1)   
                ind = torch.from_numpy(ind)
                indx.append(ind)
            self.m_ind[str(j+1)] = indx
        
        #print(len(self.m_ind['1']),len(self.m_ind['2']))
        mnn=50000  #just to find minimum number of data points in subclass of all classes
        for i in range(len(leng)):
            s  =str(i+1)
            arr = self.m_ind[s]
            for l in range(len(arr)):
                mn = len(arr[l])
                if(mn<mnn):
                    mnn= mn
                    if mnn==0:
                        print(i+1," ",l," ",arr[l])
        print("maximum support + query = ",mnn)
        #print(np.unique(labels['1']),np.unique(labels['2']))
        
    def __len__(self):
        return self.n_batch
    
    def __iter__(self):
        for i_batch in range(self.n_batch): 
            batch = [] 
            n= len(self.leng)
            rem = i_batch % n
            classes = torch.randperm(len(self.m_ind[str(rem +1)]))[:self.n_cls]  
            for c in classes:
                l = self.m_ind[str(rem +1)][c]
                pos = torch.randperm(len(l))[:self.n_per]
                batch.append(l[pos])
            batch = torch.stack(batch).t().reshape(-1)
            yield batch
            
class vt_Sampler():

    def __init__(self, label):
        
        label = np.array(label)
        indx=[]
        
        for i in (np.unique(label)): 
            ind = np.argwhere(label == i).reshape(-1)  
            ind = torch.from_numpy(ind)
            indx.append(ind)
            
        self.indx = indx
        self.length = len(indx)
        print(self.length)
    def __len__(self):
        return self.length
    
    def __iter__(self):
        for i_batch in range(self.length): 
            batch = []
            l = self.indx[i_batch]
            batch.append(l)
            batch = torch.stack(batch).t().reshape(-1)
            yield batch

## Define Models

In [5]:
def conv_block(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=0),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.MaxPool2d(2)    
    )

class ProtoNet(nn.Module):
    def __init__(self, in_channels=3, hid_channels=64, out_channels=16, c_dim=1024, o_dim=256):
        super(ProtoNet,self).__init__()
        self.common_dim = c_dim
        self.output_dim = o_dim
        self.encoder = nn.Sequential(
            conv_block(in_channels, hid_channels),
            conv_block(hid_channels, hid_channels),
            conv_block(hid_channels, hid_channels),
            conv_block(hid_channels, hid_channels),
            conv_block(hid_channels, out_channels)
        )
        
        self.dense_flowers = nn.Sequential(
            nn.Linear(c_dim, o_dim),
            nn.ReLU(),
        )
        
        self.dense_birds = nn.Sequential(
            nn.Linear(c_dim, o_dim),
            nn.ReLU(),
        )
        
        self.dense_dogs = nn.Sequential(
            nn.Linear(c_dim, o_dim),
            nn.ReLU(), 
        )
        
        self.dense_cars = nn.Sequential(
            nn.Linear(c_dim, o_dim),
            nn.ReLU(),
        )
        
        self.dense_aircrafts = nn.Sequential(
            nn.Linear(c_dim, 512),
            nn.ReLU(),
            nn.Linear(512, o_dim),
            nn.ReLU(),
        )
    
    def forward(self, x):
        x = self.encoder(x)
        z_common = x.view(x.size(0), -1)
        z_flowers = self.dense_flowers(z_common)
        z_birds = self.dense_birds(z_common)        
        z_dogs = self.dense_dogs(z_common)        
        z_cars = self.dense_cars(z_common)        
        z_planes = self.dense_aircrafts(z_common)
        
        return z_common, z_flowers, z_birds, z_dogs, z_cars, z_planes 
#         return z_birds

In [6]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

class ProtoImageNet(nn.Module):
    def __init__(self, model_name, o_dim=64, c_dim=256, feature_extract=True, is_pretrained=True):
        super(ProtoImageNet,self).__init__()
        self.model_name = model_name
        
        self.model = None
        self.common_dim = c_dim
        self.output_dim = o_dim
        
        if(model_name == 'resnet'):
            self.model = models.resnet34(pretrained=is_pretrained)
            set_parameter_requires_grad(self.model, feature_extract)
            num_features = self.model.fc.in_features
            self.model.fc = nn.Linear(num_features, c_dim)
        
        elif(model_name == 'vgg'):
            pass
        
        
        self.dense_flowers = nn.Sequential(
            nn.Linear(c_dim, o_dim),
            nn.ReLU(),
        )
        
        self.dense_birds = nn.Sequential(
            nn.Linear(c_dim, o_dim),
            nn.ReLU(),
        )
        
        self.dense_dogs = nn.Sequential(
            nn.Linear(c_dim, o_dim),
            nn.ReLU(), 
        )
        
        self.dense_cars = nn.Sequential(
            nn.Linear(c_dim, o_dim),
            nn.ReLU(),
        )
        
        self.dense_aircrafts = nn.Sequential(
            nn.Linear(c_dim, o_dim),
            nn.ReLU(),
        )
        
    def forward(self, x):
        z_common = self.model(x)
        z_flowers = self.dense_flowers(z_common)
        z_birds = self.dense_birds(z_common)        
        z_dogs = self.dense_dogs(z_common)        
        z_cars = self.dense_cars(z_common)        
        z_planes = self.dense_aircrafts(z_common)
        
        return z_common, z_flowers, z_birds, z_dogs, z_cars, z_planes 
    

## Utility Functions

In [7]:
def distance_metric(query_embeddings, prototypes):
#     print(query_embeddings.size(), prototypes.size())
    query_count = query_embeddings.size()[0]
    proto_count = prototypes.size()[0]
    
    a = query_embeddings.unsqueeze(1).expand(query_count, proto_count, -1)
    b = prototypes.unsqueeze(0).expand(query_count, proto_count, -1)
    
    logits = -((a - b)**2).sum(dim=2)
    return logits
    
    
def count_acc(logits, labels):
    print(logits.size(), labels.size())
    preds = torch.argmax(logits, dim=1)
    print(labels,'\n',preds)
    return (preds == labels).type(torch.cuda.FloatTensor).mean().item()

def count_subclass_acc(logits, labels, n_sub_classes):
    preds = torch.argmax(logits, dim=1) 
    result = torch.ones(n_sub_classes).to(device)
    
    samples_per_subclass = int(int(labels.size()[0])/n_sub_classes)
    for sc in range(n_sub_classes):
        sc_beg, sc_end = sc*samples_per_subclass, (sc+1)*samples_per_subclass 
        result[sc] = (preds[sc_beg:sc_end] == labels[sc_beg:sc_end]).type(torch.cuda.FloatTensor).mean().item()
    return result


class Averager():

    def __init__(self):
        self.n = 0
        self.v = 0

    def add(self, x):
        self.v = (self.v * self.n + x) / (self.n + 1)
        self.n += 1

    def item(self):
        return self.v
    
class Maximizer():

    def __init__(self):
        self.v = 0

    def add(self, x):
        self.v = max(self.v, x)

    def item(self):
        return self.v

In [8]:
debug_size = False
save_model = False
reload_model = True

MODEL_PATH = 'models/proto_model'+'_aircrafts_resnet34_epoch_1k_working'+'.pt'
PROTOTYPES_PATH = 'models/prototypes'+'_aircrafts_resnet34_epoch_1k_working'+'.pt'
OPTIMIZER_PATH = 'models/optimizer'+'_aircrafts_resnet34_epoch_1k_working'+'.pt'
ACCURACY_PATH = 'models/accuracy'+'_aircrafts_resnet34_epoch_1k_working'+'.pt'


In [ ]:
device = torch.device("cuda:0")

n_sub_classes = 20
support = 30
query = 10
samples_per_class = support + query # support + query exapmles
episodes = 10
alpha = 0.5

# make sure order birds, dogs, aircrafts, cars

trainset = Dataset2('train',['aircrafts'])  # list can contain 'dogs','aircrafts','cars','birds'
train_sampler = Batch_Sampler(trainset.label,trainset.leng, n_sub_classes, samples_per_class, episodes)
total_training_subclass = len(np.unique(trainset.label))

valset = Dataset2('val',['aircrafts'])
val_sampler = vt_Sampler(valset.label)

# model = ProtoNet().cuda()
net = ProtoImageNet('resnet')
model = torch.nn.DataParallel(net)
model.to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

subclass_prototypes = torch.zeros((total_subclasses, net.output_dim)).to(device)
train_accuracy = {"Birds": Averager(), "Dogs": Averager(), "Cars": Averager(), "Aircrafts": Averager()}
val_accuracy = {"Birds": Averager(), "Dogs": Averager(), "Cars": Averager(), "Aircrafts": Averager()}

if(reload_model):
    model = torch.load(MODEL_PATH)
    optimizer = torch.load(OPTIMIZER_PATH)
    subclass_prototypes = torch.load(PROTOTYPES_PATH)
    print("Model Reloaded!")

In [19]:
print(net)

ProtoImageNet(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [10]:
for n,p in model.named_parameters():
    if p.requires_grad:
         print(n)

module.model.fc.weight
module.model.fc.bias
module.dense_flowers.0.weight
module.dense_flowers.0.bias
module.dense_birds.0.weight
module.dense_birds.0.bias
module.dense_dogs.0.weight
module.dense_dogs.0.bias
module.dense_cars.0.weight
module.dense_cars.0.bias
module.dense_aircrafts.0.weight
module.dense_aircrafts.0.bias


## Training Model on Single class dataset

In [13]:
num_epochs = 1000

accuracy_dic = {i+1:Averager() for i in range(total_training_subclass)}

for epoch in range(num_epochs):
#     print('===============epoch================ ',epoch)
#     lr_scheduler.step()
    
    is_sampled = torch.zeros(total_training_subclass).to(device)
    
    model.train()
    print("Training:")
    train_loader = DataLoader(dataset=trainset, batch_sampler=train_sampler, pin_memory=True, num_workers=8)
    
    # each batch is an episode consisting of one class with 'n_sub_classes' number of fine classes in that class.
    for i_batch, batch in enumerate(train_loader):
        start = time.time()
        print('=============epoch {0:2d} & batch {1:2d}============ '.format(epoch+1,i_batch+1))
        x, y = batch
        
        if(debug_size):
            print("\n\nBatch size from data : ",x.size(), y.size())
            print(y)
        
        coarse_class = coarseclass_name(y[0])
    
        x = x.to(device).reshape((samples_per_class, n_sub_classes, img_channels, img_height, img_width)).permute(1, 0, 2, 3, 4)
        y = y.to(device).reshape((samples_per_class, n_sub_classes)).permute(1, 0)
        # x/y is now n_sub_classes * samples_per_class * image/label
        
        if(debug_size):
            print("\n\nBatch size after 5-D stacking : ", x.size(), y.size())
            print(y)

        x_support = x[:,:support,:,:,:].reshape(n_sub_classes*support,img_channels, img_height, img_width)        
        x_query = x[:,support:,:,:,:].reshape(n_sub_classes*query,img_channels, img_height, img_width)
        y_support = y[:,:support].reshape(n_sub_classes*support)  
        y_query = y[:,support:].reshape(n_sub_classes*query)
        #x_support/y_support is now n_sub_classes * support * image/label
        
        if(debug_size):
            print("\n\nBatch support size of model input : ",x_support.size(), y_support.size())
            print(y_support)
            print("Batch query size of model input : ",x_query.size(), y_query.size())
            print(y_query)
        
        _, _, _, _, _, z_support  = model(x_support)
        _, _, _, _, _,   z_query  = model(x_query)
        
       
        
        z_support = z_support.reshape((n_sub_classes, support, -1))
        prototypes = torch.mean(z_support, dim=1)
        # prototypes dim is n_sub_classes * latent_len
        
        if(debug_size):
            print("\n\nSize of batch prototypes : ",prototypes.size())
        
        # update global subclass prototypes - subclasses are labels 1-n but prototype array is indexed 0-n-1
        for l in range(n_sub_classes):
            subclass_prototypes[y_support[l*support]-1] = alpha*subclass_prototypes[y_support[l*support]-1] + (1-alpha)*prototypes[l]   #subclass_prototypes are 0 indexed but y is 1 indexed
            
        
        logits = distance_metric(z_query, prototypes)
        labels = torch.Tensor([i for i in range(n_sub_classes)]).unsqueeze(1).expand(n_sub_classes, query).reshape(-1).type(torch.cuda.LongTensor)
        # creating labels tensor to calculate loss
        
        if(debug_size):
            print("\n\nBatch query logits size : ",logits.size())
            print("Batch query labels size & labels : ",labels.size())
            print(labels)
            
        loss = F.cross_entropy(logits, labels)
        
#         batch_all_accuracy = count_acc(logits, labels)
#         print(batch_all_accuracy)
        batch_subclass_accuracy = count_subclass_acc(logits, labels, n_sub_classes)
        print(batch_subclass_accuracy)
        
        # to estimate how many subclasses are sampled over episodes - for single class data only
        # update each subclass accuracy
        for sub_class in range(n_sub_classes):
            is_sampled[y[sub_class][0]-1] = 1
            accuracy_dic[int(y[sub_class][0])].add(batch_subclass_accuracy[sub_class])
            # update the sub_class accuracy in accuracy dictionary
            
#         print(acc)
        
    
        # update the model parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        prototypes = None; logits = None; loss = None
        
        # printing batch stats
        if((i_batch+1)%1==0):
            for key in accuracy_dic.keys():
                print("{0:2d} : {1:.2f}".format(key, accuracy_dic[key].item()))
            
#             print("Episode : {0:2d} - Loss : {1:.5f} - Accuracy : {2:.2f}".format(i_batch, loss, batch_accuracy))    
            print("{0:.0f} subclasses explored out of {1:2d} subclasses".format(torch.sum(is_sampled), total_training_subclass))
        
        print("Episode Time : ",time.time()-start)
        debug_size = False

if(save_model):
    # save model
    torch.save(model, MODEL_PATH)
    torch.save(subclass_prototypes, PROTOTYPES_PATH)
    torch.save(optimizer, OPTIMIZER_PATH)
    torch.save(accuracy_dic, ACCURACY_PATH)

Training:
=============epoch  1 & batch  1============ 
tensor([0.6000, 0.4000, 0.2000, 0.0000, 0.2000, 0.2000, 0.3000, 0.3000, 0.1000,
        0.6000, 0.0000, 0.1000, 0.3000, 0.7000, 0.3000, 0.5000, 0.0000, 0.3000,
        0.3000, 0.1000], device='cuda:0')
 1 : 0.30
 2 : 0.20
 3 : 0.30
 4 : 0.00
 5 : 0.00
 6 : 0.10
 7 : 0.50
 8 : 0.00
 9 : 0.00
10 : 0.00
11 : 0.00
12 : 0.00
13 : 0.00
14 : 0.00
15 : 0.00
16 : 0.00
17 : 0.00
18 : 0.00
19 : 0.00
20 : 0.00
21 : 0.00
22 : 0.20
23 : 0.00
24 : 0.00
25 : 0.00
26 : 0.00
27 : 0.00
28 : 0.00
29 : 0.00
30 : 0.30
31 : 0.00
32 : 0.70
33 : 0.00
34 : 0.00
35 : 0.00
36 : 0.00
37 : 0.60
38 : 0.00
39 : 0.20
40 : 0.00
41 : 0.00
42 : 0.00
43 : 0.00
44 : 0.30
45 : 0.00
46 : 0.30
47 : 0.60
48 : 0.00
49 : 0.10
50 : 0.00
51 : 0.10
52 : 0.00
53 : 0.00
54 : 0.00
55 : 0.40
56 : 0.00
57 : 0.00
58 : 0.00
59 : 0.00
60 : 0.00
61 : 0.00
62 : 0.00
63 : 0.00
64 : 0.00
65 : 0.00
66 : 0.00
67 : 0.00
68 : 0.00
69 : 0.30
70 : 0.00
20 subclasses explored out of 70 subclasse

=============epoch  1 & batch  9============ 
tensor([0.2000, 0.3000, 0.4000, 0.1000, 0.4000, 0.1000, 0.1000, 0.4000, 0.1000,
        0.4000, 0.2000, 0.0000, 0.4000, 0.2000, 0.4000, 0.2000, 0.6000, 0.2000,
        0.2000, 0.5000], device='cuda:0')
 1 : 0.17
 2 : 0.35
 3 : 0.30
 4 : 0.10
 5 : 0.10
 6 : 0.25
 7 : 0.23
 8 : 0.05
 9 : 0.00
10 : 0.17
11 : 0.50
12 : 0.20
13 : 0.37
14 : 0.30
15 : 0.14
16 : 0.15
17 : 0.07
18 : 0.40
19 : 0.20
20 : 0.17
21 : 0.30
22 : 0.20
23 : 0.25
24 : 0.10
25 : 0.35
26 : 0.40
27 : 0.00
28 : 0.23
29 : 0.15
30 : 0.32
31 : 0.43
32 : 0.28
33 : 0.17
34 : 0.80
35 : 0.35
36 : 0.40
37 : 0.63
38 : 0.10
39 : 0.17
40 : 0.13
41 : 0.13
42 : 0.11
43 : 0.43
44 : 0.43
45 : 0.50
46 : 0.25
47 : 0.60
48 : 0.15
49 : 0.18
50 : 0.30
51 : 0.15
52 : 0.20
53 : 0.40
54 : 0.00
55 : 0.30
56 : 0.15
57 : 0.00
58 : 0.20
59 : 0.20
60 : 0.22
61 : 0.08
62 : 0.50
63 : 0.50
64 : 0.23
65 : 0.30
66 : 0.60
67 : 0.30
68 : 0.20
69 : 0.25
70 : 0.20
66 subclasses explored out of 70 subclasses
Episode 

tensor([0.4000, 0.7000, 0.3000, 0.1000, 0.4000, 0.4000, 0.1000, 0.4000, 0.4000,
        0.3000, 0.0000, 0.3000, 0.3000, 0.2000, 0.1000, 0.1000, 0.1000, 0.2000,
        0.1000, 0.1000], device='cuda:0')
 1 : 0.15
 2 : 0.23
 3 : 0.32
 4 : 0.12
 5 : 0.16
 6 : 0.23
 7 : 0.16
 8 : 0.10
 9 : 0.55
10 : 0.22
11 : 0.42
12 : 0.32
13 : 0.28
14 : 0.25
15 : 0.12
16 : 0.20
17 : 0.06
18 : 0.33
19 : 0.25
20 : 0.22
21 : 0.30
22 : 0.25
23 : 0.23
24 : 0.27
25 : 0.32
26 : 0.40
27 : 0.23
28 : 0.17
29 : 0.13
30 : 0.30
31 : 0.38
32 : 0.28
33 : 0.14
34 : 0.80
35 : 0.45
36 : 0.43
37 : 0.53
38 : 0.17
39 : 0.23
40 : 0.18
41 : 0.15
42 : 0.13
43 : 0.32
44 : 0.43
45 : 0.52
46 : 0.23
47 : 0.43
48 : 0.20
49 : 0.20
50 : 0.40
51 : 0.20
52 : 0.21
53 : 0.27
54 : 0.40
55 : 0.32
56 : 0.12
57 : 0.22
58 : 0.10
59 : 0.18
60 : 0.20
61 : 0.14
62 : 0.50
63 : 0.46
64 : 0.24
65 : 0.34
66 : 0.60
67 : 0.40
68 : 0.25
69 : 0.20
70 : 0.25
63 subclasses explored out of 70 subclasses
Episode Time :  3.5810773372650146
=============epoch 

tensor([0.4000, 0.1000, 0.5000, 0.4000, 0.1000, 0.1000, 0.1000, 0.1000, 0.3000,
        0.3000, 0.4000, 0.2000, 0.3000, 0.1000, 0.3000, 0.0000, 0.1000, 0.1000,
        0.3000, 0.1000], device='cuda:0')
 1 : 0.11
 2 : 0.24
 3 : 0.32
 4 : 0.10
 5 : 0.16
 6 : 0.20
 7 : 0.18
 8 : 0.10
 9 : 0.47
10 : 0.19
11 : 0.44
12 : 0.30
13 : 0.27
14 : 0.27
15 : 0.16
16 : 0.24
17 : 0.06
18 : 0.33
19 : 0.25
20 : 0.25
21 : 0.33
22 : 0.19
23 : 0.23
24 : 0.23
25 : 0.31
26 : 0.41
27 : 0.22
28 : 0.23
29 : 0.13
30 : 0.31
31 : 0.38
32 : 0.28
33 : 0.13
34 : 0.70
35 : 0.46
36 : 0.48
37 : 0.53
38 : 0.17
39 : 0.23
40 : 0.14
41 : 0.17
42 : 0.13
43 : 0.36
44 : 0.40
45 : 0.43
46 : 0.20
47 : 0.35
48 : 0.24
49 : 0.23
50 : 0.38
51 : 0.20
52 : 0.25
53 : 0.34
54 : 0.43
55 : 0.34
56 : 0.13
57 : 0.18
58 : 0.09
59 : 0.17
60 : 0.18
61 : 0.16
62 : 0.54
63 : 0.46
64 : 0.24
65 : 0.36
66 : 0.60
67 : 0.37
68 : 0.27
69 : 0.25
70 : 0.23
52 subclasses explored out of 70 subclasses
Episode Time :  1.4873285293579102
=============epoch 

tensor([0.2000, 0.0000, 0.5000, 0.3000, 0.3000, 0.5000, 0.1000, 0.0000, 0.5000,
        0.1000, 0.3000, 0.2000, 0.3000, 0.0000, 0.2000, 0.3000, 0.2000, 0.2000,
        0.1000, 0.5000], device='cuda:0')
 1 : 0.11
 2 : 0.20
 3 : 0.28
 4 : 0.14
 5 : 0.18
 6 : 0.21
 7 : 0.15
 8 : 0.13
 9 : 0.42
10 : 0.20
11 : 0.43
12 : 0.29
13 : 0.29
14 : 0.27
15 : 0.13
16 : 0.23
17 : 0.13
18 : 0.32
19 : 0.23
20 : 0.29
21 : 0.32
22 : 0.19
23 : 0.21
24 : 0.23
25 : 0.31
26 : 0.33
27 : 0.22
28 : 0.22
29 : 0.12
30 : 0.29
31 : 0.36
32 : 0.26
33 : 0.14
34 : 0.70
35 : 0.50
36 : 0.43
37 : 0.53
38 : 0.17
39 : 0.24
40 : 0.16
41 : 0.17
42 : 0.13
43 : 0.39
44 : 0.44
45 : 0.43
46 : 0.21
47 : 0.30
48 : 0.22
49 : 0.27
50 : 0.34
51 : 0.20
52 : 0.30
53 : 0.31
54 : 0.53
55 : 0.30
56 : 0.14
57 : 0.13
58 : 0.08
59 : 0.13
60 : 0.19
61 : 0.17
62 : 0.51
63 : 0.43
64 : 0.22
65 : 0.33
66 : 0.59
67 : 0.42
68 : 0.31
69 : 0.29
70 : 0.21
44 subclasses explored out of 70 subclasses
Episode Time :  1.4623157978057861
=============epoch 

...

tensor([0.2000, 0.4000, 0.1000, 0.3000, 0.4000, 0.2000, 0.3000, 0.5000, 0.2000,
        0.1000, 0.1000, 0.5000, 0.2000, 0.1000, 0.3000, 0.3000, 0.1000, 0.3000,
        0.1000, 0.5000], device='cuda:0')
 1 : 0.14
 2 : 0.21
 3 : 0.22
 4 : 0.26
 5 : 0.16
 6 : 0.22
 7 : 0.18
 8 : 0.20
 9 : 0.40
10 : 0.20
11 : 0.38
12 : 0.22
13 : 0.27
14 : 0.20
15 : 0.15
16 : 0.21
17 : 0.12
18 : 0.28
19 : 0.25
20 : 0.27
21 : 0.38
22 : 0.24
23 : 0.17
24 : 0.24
25 : 0.28
26 : 0.31
27 : 0.23
28 : 0.25
29 : 0.10
30 : 0.29
31 : 0.36
32 : 0.31
33 : 0.16
34 : 0.69
35 : 0.43
36 : 0.47
37 : 0.45
38 : 0.18
39 : 0.27
40 : 0.18
41 : 0.19
42 : 0.11
43 : 0.32
44 : 0.48
45 : 0.47
46 : 0.30
47 : 0.31
48 : 0.23
49 : 0.23
50 : 0.32
51 : 0.25
52 : 0.31
53 : 0.29
54 : 0.57
55 : 0.34
56 : 0.17
57 : 0.16
58 : 0.10
59 : 0.17
60 : 0.19
61 : 0.16
62 : 0.45
63 : 0.40
64 : 0.24
65 : 0.33
66 : 0.48
67 : 0.50
68 : 0.26
69 : 0.26
70 : 0.17
69 subclasses explored out of 70 subclasses
Episode Time :  2.5278210639953613
=============epoch 

tensor([0.1000, 0.3000, 0.4000, 0.2000, 0.1000, 0.2000, 0.4000, 0.3000, 0.2000,
        0.0000, 0.3000, 0.5000, 0.6000, 0.4000, 0.8000, 0.4000, 0.1000, 0.1000,
        0.2000, 0.4000], device='cuda:0')
 1 : 0.15
 2 : 0.21
 3 : 0.22
 4 : 0.25
 5 : 0.16
 6 : 0.22
 7 : 0.18
 8 : 0.20
 9 : 0.40
10 : 0.20
11 : 0.38
12 : 0.22
13 : 0.27
14 : 0.20
15 : 0.15
16 : 0.21
17 : 0.12
18 : 0.29
19 : 0.25
20 : 0.27
21 : 0.38
22 : 0.24
23 : 0.17
24 : 0.24
25 : 0.28
26 : 0.31
27 : 0.23
28 : 0.25
29 : 0.10
30 : 0.30
31 : 0.36
32 : 0.31
33 : 0.16
34 : 0.69
35 : 0.43
36 : 0.47
37 : 0.45
38 : 0.18
39 : 0.27
40 : 0.18
41 : 0.19
42 : 0.11
43 : 0.32
44 : 0.48
45 : 0.47
46 : 0.30
47 : 0.30
48 : 0.23
49 : 0.23
50 : 0.32
51 : 0.25
52 : 0.31
53 : 0.29
54 : 0.57
55 : 0.34
56 : 0.17
57 : 0.16
58 : 0.10
59 : 0.18
60 : 0.18
61 : 0.16
62 : 0.45
63 : 0.40
64 : 0.24
65 : 0.33
66 : 0.48
67 : 0.50
68 : 0.26
69 : 0.26
70 : 0.17
67 subclasses explored out of 70 subclasses
Episode Time :  3.271993637084961
=============epoch 1

tensor([0.1000, 0.4000, 0.4000, 0.2000, 0.3000, 0.4000, 0.1000, 0.1000, 0.1000,
        0.3000, 0.2000, 0.3000, 0.2000, 0.2000, 0.4000, 0.2000, 0.4000, 0.0000,
        0.0000, 0.1000], device='cuda:0')
 1 : 0.15
 2 : 0.21
 3 : 0.22
 4 : 0.25
 5 : 0.16
 6 : 0.22
 7 : 0.18
 8 : 0.20
 9 : 0.40
10 : 0.20
11 : 0.38
12 : 0.22
13 : 0.27
14 : 0.20
15 : 0.15
16 : 0.21
17 : 0.12
18 : 0.29
19 : 0.25
20 : 0.27
21 : 0.38
22 : 0.24
23 : 0.17
24 : 0.24
25 : 0.28
26 : 0.31
27 : 0.23
28 : 0.25
29 : 0.10
30 : 0.30
31 : 0.36
32 : 0.31
33 : 0.16
34 : 0.69
35 : 0.43
36 : 0.47
37 : 0.45
38 : 0.18
39 : 0.27
40 : 0.18
41 : 0.19
42 : 0.11
43 : 0.32
44 : 0.48
45 : 0.47
46 : 0.30
47 : 0.30
48 : 0.23
49 : 0.23
50 : 0.32
51 : 0.25
52 : 0.31
53 : 0.29
54 : 0.57
55 : 0.34
56 : 0.17
57 : 0.16
58 : 0.10
59 : 0.18
60 : 0.18
61 : 0.16
62 : 0.45
63 : 0.40
64 : 0.24
65 : 0.33
66 : 0.47
67 : 0.50
68 : 0.26
69 : 0.26
70 : 0.17
59 subclasses explored out of 70 subclasses
Episode Time :  1.6474928855895996
=============epoch 

tensor([0.2000, 0.3000, 0.4000, 0.2000, 0.4000, 0.2000, 0.2000, 0.3000, 0.2000,
        0.3000, 0.4000, 0.2000, 0.7000, 0.3000, 0.2000, 0.1000, 0.3000, 0.1000,
        0.1000, 0.5000], device='cuda:0')
 1 : 0.15
 2 : 0.21
 3 : 0.22
 4 : 0.25
 5 : 0.16
 6 : 0.22
 7 : 0.18
 8 : 0.20
 9 : 0.40
10 : 0.20
11 : 0.38
12 : 0.22
13 : 0.27
14 : 0.20
15 : 0.15
16 : 0.21
17 : 0.12
18 : 0.29
19 : 0.25
20 : 0.27
21 : 0.38
22 : 0.24
23 : 0.17
24 : 0.24
25 : 0.28
26 : 0.31
27 : 0.23
28 : 0.25
29 : 0.10
30 : 0.30
31 : 0.36
32 : 0.31
33 : 0.16
34 : 0.69
35 : 0.43
36 : 0.47
37 : 0.45
38 : 0.18
39 : 0.27
40 : 0.18
41 : 0.19
42 : 0.11
43 : 0.32
44 : 0.48
45 : 0.47
46 : 0.30
47 : 0.30
48 : 0.23
49 : 0.23
50 : 0.32
51 : 0.25
52 : 0.31
53 : 0.29
54 : 0.57
55 : 0.34
56 : 0.17
57 : 0.16
58 : 0.10
59 : 0.18
60 : 0.18
61 : 0.16
62 : 0.45
63 : 0.40
64 : 0.24
65 : 0.33
66 : 0.47
67 : 0.50
68 : 0.26
69 : 0.26
70 : 0.18
46 subclasses explored out of 70 subclasses
Episode Time :  1.410979986190796
=============epoch 1

KeyboardInterrupt: 

## Best accuracy results obtained

Aircraft : 54.8% (using conv_blocks with resnet34)
Birds :  48.2% (using conv_blocks with resnet34)
Dogs : 38.4% (very less trained)
Cars : 31.2% (very less trained)

In [ ]:
# def count_subclass_acc(logits, labels, n_sub_classes):
#     preds = torch.argmax(logits, dim=1) 
#     result = torch.ones(n_sub_classes).to(device)
    
#     samples_per_subclass = int(int(labels.size()[0])/n_sub_classes)
#     for sc in range(n_sub_classes):
#         sc_beg, sc_end = sc*samples_per_subclass, (sc+1)*samples_per_subclass 
#         result[sc] = (preds[sc_beg:sc_end] == labels[sc_beg:sc_end]).type(torch.cuda.FloatTensor).mean().item()
#     return result


## Testing Model on "Aircrafts" dataset

In [ ]:
# ################################################################################################
# #
# #
# #  Validating Model
# #
# #

# model.eval()
# print("Validation:")
# val_loader = DataLoader(dataset=valset, batch_sampler=val_sampler, pin_memory=True, num_workers=8)
# val_accuracy_dic = {i+1:Averager() for i in range(total_training_subclass)}

# for j, batch in enumerate(val_loader):
#     x, y = batch
#     x = x.to(device)
#     y = y.to(device)

#     labels = y

#     _, _, _, _, _, z  = model(x)

#     logits = distance_metric(z, subclass_prototypes)

#     if(debug_size):
#         print("\n\nValidation Logits size and Labels size",logits.size(), labels.size())

#     loss = F.cross_entropy(logits, labels)
#     val_batch_accuracy = count_acc(logits, labels)
#     val_accuracy_dic[int(y[0])].add(val_batch_accuracy)
    
# for key in val_accuracy_dic.keys():
#     print("{0:2d} - {1:.2f}".format(key, val_accuracy_dic[key].item()))


## Training Models on all class dataset

In [ ]:
# debug_size = False

# num_epochs = 1000
# total_training_subclass = len(np.unique(trainset.label))

# accuracy_dic = {i+1:Averager() for i in range(total_training_subclass)}

# for epoch in range(num_epochs):
# #     print('===============epoch================ ',epoch)
# #     lr_scheduler.step()
    
#     is_sampled = torch.zeros(total_training_subclass).to(device)
    
#     model.train()
#     print("Training:")
#     train_loader = DataLoader(dataset=trainset, batch_sampler=train_sampler, pin_memory=True, num_workers=8)
    
#     # each batch is an episode consisting of one class with 'n_sub_classes' number of fine classes in that class.
#     start = time.time()
#     for i_batch, batch in enumerate(train_loader):
#         print('=============epoch {0:2d} & batch {1:2d}============ '.format(epoch+1,i_batch+1))
#         x, y = batch
        
#         if(debug_size):
#             print("\n\nBatch size from data : ",x.size(), y.size())
#             print(y)
        
#         coarse_class = coarseclass_name(y[0])
        
#         if(not coarse_class=='Aircrafts'):
#             continue
        
        
#         x = x.to(device).reshape((samples_per_class, n_sub_classes, img_channels, img_height, img_width)).permute(1, 0, 2, 3, 4)
#         y = y.to(device).reshape((samples_per_class, n_sub_classes)).permute(1, 0)
#         # x/y is now n_sub_classes * samples_per_class * image/label
        
#         if(debug_size):
#             print("\n\nBatch size after 5-D stacking : ", x.size(), y.size())
#             print(y)

#         x_support = x[:,:support,:,:,:].reshape(n_sub_classes*support,img_channels, img_height, img_width)        
#         x_query = x[:,support:,:,:,:].reshape(n_sub_classes*query,img_channels, img_height, img_width)
#         y_support = y[:,:support].reshape(n_sub_classes*support)  
#         y_query = y[:,support:].reshape(n_sub_classes*query)
#         #x_support/y_support is now n_sub_classes * support * image/label
        
#         if(debug_size):
#             print("\n\nBatch support size of model input : ",x_support.size(), y_support.size())
#             print(y_support)
#             print("Batch query size of model input : ",x_query.size(), y_query.size())
#             print(y_query)
        
        
#         if(coarse_class=='Flowers'):
#             _, z_support, _, _, _, _  = model(x_support)
#             _,   z_query, _, _, _, _  = model(x_query)
            
#         elif(coarse_class=='Birds'):
#             _, _, z_support, _, _, _  = model(x_support)
#             _, _,   z_query, _, _, _  = model(x_query)
        
#         elif(coarse_class=='Dogs'):
#             _, _, _, z_support, _, _  = model(x_support)
#             _, _, _,   z_query, _, _  = model(x_query)
            
#         elif(coarse_class=='Cars'):
#             _, _, _, _, z_support, _  = model(x_support)
#             _, _, _, _,   z_query, _  = model(x_query)
        
#         elif(coarse_class=='Aircrafts'):
#             _, _, _, _, _, z_support  = model(x_support)
#             _, _, _, _, _,   z_query  = model(x_query)
        
       
        
#         z_support = z_support.reshape((n_sub_classes, support, -1))
#         prototypes = torch.mean(z_support, dim=1)
#         # prototypes dim is n_sub_classes * latent_len
        
#         if(debug_size):
#             print("\n\nSize of batch prototypes : ",prototypes.size())
        
#         # update global subclass prototypes - subclasses are labels 1-n but prototype array is indexed 0-n-1
#         for l in range(n_sub_classes):
#             subclass_prototypes[y_support[l*support]-1] = alpha*subclass_prototypes[y_support[l*support]-1] + (1-alpha)*prototypes[l]   #subclass_prototypes are 0 indexed but y is 1 indexed
            
        
#         logits = distance_metric(z_query, prototypes)
#         labels = torch.Tensor([i for i in range(n_sub_classes)]).unsqueeze(1).expand(n_sub_classes, query).reshape(-1).type(torch.cuda.LongTensor)
#         # creating labels tensor to calculate loss
        
#         if(debug_size):
#             print("\n\nBatch query logits size : ",logits.size())
#             print("Batch query labels size & labels : ",labels.size())
#             print(labels)
            
#         loss = F.cross_entropy(logits, labels)
        
#         batch_accuracy = count_acc(logits, labels)
        
        
# #         train_accuracy[coarse_class].add(batch_accurracy)   # required for multiple classes
        
#         # to estimate how many subclasses are sampled over episodes - for single class data only
#         for sub_class in range(n_sub_classes):
#             is_sampled[y[sub_class][0]-1] = 1
#             accuracy_dic[int(y[sub_class][0])].add(batch_accuracy)
#             # update the sub_class accuracy in accuracy dictionary
            
# #         print(acc)
        
    
#         # update the model parameters
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         prototypes = None; logits = None; loss = None
        
#         # printing batch stats
#         if((i_batch+1)%1==0):
#             for key in accuracy_dic.keys():
#                 print("{0:2d} : {1:.2f}".format(key, accuracy_dic[key].item()))
            
# #             print("Episode : {0:2d} - Loss : {1:.5f} - Accuracy : {2:.2f}".format(i_batch, loss, batch_accuracy))    
#             print("{0:.0f} subclasses explored out of {1:2d} subclasses".format(torch.sum(is_sampled), total_training_subclass))

#         debug_size = False



# #     model.eval()
# #     print("Validation:")
# #     val_loader = DataLoader(dataset=valset, batch_sampler=val_sampler, pin_memory=True, num_workers=8)
    
# #     if(not i%10==0):
# #         continue
        
# #     for j, batch in enumerate(val_loader):
# #         x, y = batch
# #         coarse_class = coarseclass_name(y[0])
        
# #         if(not coarse_class=='Aircrafts'):
# #             continue
            
# #         x = x.to(device)
# #         y = y.to(device)
        
# #         labels = y
        
# #         if(coarse_class=='Flowers'):
# #             _, z, _, _, _, _  = model(x)
            
# #         elif(coarse_class=='Birds'):
# #             _, _, z, _, _, _  = model(x)
        
# #         elif(coarse_class=='Dogs'):
# #             _, _, _, z, _, _  = model(x)
            
# #         elif(coarse_class=='Cars'):
# #             _, _, _, _, z, _  = model(x)
        
# #         elif(coarse_class=='Aircrafts'):
# #             _, _, _, _, _, z  = model(x)
    
# #         logits = distance_metric(z, subclass_prototypes)
# # #         print(logits.size(), labels.size())
# #         loss = F.cross_entropy(logits, labels)
# #         acc = count_acc(logits, labels)
# # #         print(acc)
# #         val_accuracy[coarse_class].add(acc)
        
# #     print("Time taken : ",time.time()-start)
# #         print("{0:.3f} {1:.3f}".format(train_accuracy['Aircrafts'].item(), val_accuracy['Aircrafts'].item()))
# #     for i, key in enumerate(train_accuracy.keys()):
# #         print("{0:10s} - {1:.2f} ".format(key, train_accuracy[key].item()))
        

# # save model
# torch.save(model, MODEL_PATH)
# torch.save(subclass_prototypes, PROTOTYPES_PATH)

In [ ]:
# MODEL_PATH = 'models/proto_model'+'_aircrafts_resnet34_epoch_1k'+'.pt'
# PROTOTYPES_PATH = 'models/prototypes'+'_aircrafts_resnet34_epoch_1k'+'.pt'
# OPTIMIZER_PATH = 'models/optimizer'+'_aircrafts_resnet34_epoch_1k'+'.pt'
# ACCURACY_PATH = 'models/accuracy'+'_aircrafts_resnet34_epoch_1k'+'.pt'

# torch.save(model, MODEL_PATH)
# torch.save(subclass_prototypes, PROTOTYPES_PATH)
# torch.save(optimizer, OPTIMIZER_PATH)
# torch.save(accuracy_dic, ACCURACY_PATH)

In [ ]:
# print(x.size(), y.size())

# x1 = x.reshape(x.size()[0]*x.size()[1],img_channels,img_height,img_width)
# y1 = y.reshape(x.size()[0]*x.size()[1])

# # print(x1.size(), y1.size())

# x2 = x1.reshape(x.size()[0], x.size()[1],img_channels,img_height,img_width)
# y2 = y1.reshape(x.size()[0], x.size()[1])

# # print(x2.size(), y2.size())

# # # print(y, y2)
# # # print(torch.equal(x,x2))


# z1 = model(x1)[5]
# print(z1.size())

# z2 = z1.reshape(x.size()[0], x.size()[1], -1)
# print(z2.size())


# z1_one = model(x1[0].reshape(1,img_channels,img_height,img_width))[5]
# z2_one = model(x1)[5]

# print(z1_one[0].size(),z2_one[0].size())
# print(torch.equal(z1_one[0],z2_one[0]))

In [ ]:
# print(z1.size())
# # print(z1[0])

# zl = model(x1[0].reshape(1,img_channels,img_height,img_width))[5]

# for i in range(z1.size()[0]):
#     print(torch.equal(zl[0],z1[i]))
    
# # print(zl[0])

In [ ]:
# class ResNet(nn.Module):
#     def __init__(self, o_dim=64, c_dim=5, feature_extract=True, is_pretrained=True):
#         super(ResNet,self).__init__()
        
#         self.model = nn.Sequential(
#                         nn.Conv2d(3, 5, 3),
#                     )
        
#     def forward(self, x):
#         z = self.model(x)
#         z = z.view(z.size(0), -1)

#         return z
        

In [ ]:
# my_net = ResNet().to(device)

# print(x1.size(), x1[0].size())

# y1 = my_net(x1)
# print(y.size())

# x1_prime = x1[0].reshape(1,img_channels,img_height,img_width)
# y1_prime = my_net(x1_prime)
# print(y1_prime.size())

# print(y1[0].size(), y1_prime[0].size())
# print(torch.equal(y1[0],y1_prime[0]))

In [ ]:
# j1 = x1[0:2,:,:,:]
# j2 = x1[1:3,:,:,:]

# print(j1.size(), j2.size())
# if(torch.equal(j1[1],j2[0])):
#     print("j1[1] & j2[0] are equal")

# k1 = my_net(j1)
# k2 = my_net(j2)

# print(k1.size(), k2.size())
# print(torch.equal(k1[1], k2[0]))
# print("Something is wrong here!")
# print(k1,'\n',k2)
# print("*"*100)
# j3 = j1[1].reshape(1,img_channels,img_height,img_width)
# j4 = j2[0].reshape(1,img_channels,img_height,img_width)
# l1 = my_net(j3)
# l2 = my_net(j4)

# print(torch.equal(j3,j4))
# print(l1.size(), l2.size())
# print(torch.equal(l1, l2))

In [ ]:
# model.eval()
# print("Validation:")

# for j, batch in enumerate(val_loader):
#     x, y = batch
#     coarse_class = coarseclass_name(y[0])

#     if(not coarse_class=='Aircrafts'):
#         continue

#     x = x.to(device)
#     y = y.to(device)

#     labels = y

#     if(coarse_class=='Flowers'):
#         _, z, _, _, _, _  = model(x)

#     elif(coarse_class=='Birds'):
#         _, _, z, _, _, _  = model(x)

#     elif(coarse_class=='Dogs'):
#         _, _, _, z, _, _  = model(x)

#     elif(coarse_class=='Cars'):
#         _, _, _, _, z, _  = model(x)

#     elif(coarse_class=='Aircrafts'):
#         _, _, _, _, _, z  = model(x)

#     logits = distance_metric(z, subclass_prototypes)
# #         print(logits.size(), labels.size())
#     loss = F.cross_entropy(logits, labels)
#     acc = count_acc(logits, labels)
#     print(acc)
#     val_accuracy[coarse_class].add(acc)

In [ ]:
# alpha=0
#     epoch = 0
#         Time taken :  7.636490106582642
#         Flowers    - 0.45 
#         Birds      - 0.16 
#         Dogs       - 0.16 
#         Cars       - 0.12 
#         Aircrafts  - 0.29

#     epoch =  150
#         Time taken :  10.503242015838623
#         Flowers    - 0.41 
#         Birds      - 0.20 
#         Dogs       - 0.17 
#         Cars       - 0.11 
#         Aircrafts  - 0.24
        
# alpha=0.5
#     epoch = 0
#         Time taken :  13.722722053527832
#         Flowers    - 0.39 
#         Birds      - 0.22 
#         Dogs       - 0.16 
#         Cars       - 0.14 
#         Aircrafts  - 0.27


## Testing Data

In [ ]:
# model.eval()

# val_acc = Averager()

# for j, batch in enumerate(val_loader):
#     x, y = batch
#     x = x.to(device)
#     y = y.to(device)

#     coarse_class = coarseclass_name(y[0])
#     fine_class = subclass_name(y[0])
# #     print(coarse_class, fine_class, y[0])

#     if(coarse_class=='Flowers'):
#         _, z, _, _, _, _  = model(x)

#     elif(coarse_class=='Birds'):
#         _, _, z, _, _, _  = model(x)

#     elif(coarse_class=='Dogs'):
#         _, _, _, z, _, _  = model(x)

#     elif(coarse_class=='Cars'):
#         _, _, _, _, z, _  = model(x)

#     elif(coarse_class=='Aircrafts'):
#         _, _, _, _, _, z  = model(x)

#     logits = distance_metric(z, subclass_prototypes)

#     loss = F.cross_entropy(logits, y)  # y has the correct labels
#     acc = count_acc(logits, y)
#     val_acc.add(acc)
#     print(acc, val_acc.item())

In [ ]:
# valset = Dataset2('val')
# val_sampler = vt_Sampler(valset.label)
# val_loader = DataLoader(dataset=valset, batch_sampler=val_sampler)

# tsum = 0
# for batch in val_loader:
#     x, y = batch
#     print(x.size(),y.size())
#     tsum += list(y.size())[0]

## Visualize graph

In [20]:
# # install graphviz in linux
# # sudo apt-get install graphviz
# # install torchviz in python
# # pip install torchviz

# from torchviz import make_dot
# from torch.autograd import Variable
# import torch 
# import torch.nn as nn

# my_model = nn.Sequential()
# my_model.add_module('W0', nn.Linear(8,16))
# my_model.add_module('tanh', nn.Tanh())
# my_model.add_module('W1', nn.Linear(16,1))


# x1 = Variable(torch.randn(1,8))
# y1 = my_model(x1)

# dot = make_dot(y1.mean(), params=dict(my_model.named_parameters()))
# dot.format = 'png'
# dot.render()

ExecutableNotFound: failed to execute ['dot', '-Tpng', '-O', 'Digraph.gv'], make sure the Graphviz executables are on your systems' PATH

## Display episodes samples

In [ ]:
#         # zero^th dimension has samples from different sampled subclass of a class
#         # diplaying support and query element of 1st subclass of a class
#         for k in range(x.size()[1]):
#             z = x[0][k]
#             z = trainset.invTrans(z).permute(2,1,0)
#             print(subclass_name(y[0][k]), coarseclass_name(y[0][k]))
#             plt.imshow(z)
#             plt.pause(0.01)
#             break